In [1]:
import json
entity2id = json.load(open('tgredial_bert/entity2id.json', 'r'))
movie_ids = json.load(open('tgredial_bert/movie_ids.json', 'r'))

In [ ]:
entity2id

In [2]:
id2entity = {v: k for k, v in entity2id.items()}

In [3]:
movie_list = []
for key in id2entity.keys():
    if int(key) in movie_ids:
        movie_list.append(id2entity[key])

In [4]:
len(movie_list)

31985

In [14]:
int(len(movie_list)/3) * 3

31983

In [16]:
import random
import numpy as np

random.shuffle(movie_list)
movies = np.asarray(movie_list[:(int(len(movie_list)/3)*3)]).reshape(int(len(movie_list)/3), 3)

### Sample movies for dialog generation

In [17]:
movies[:10]

array([['心魔（1937年陈武扬执导电影）', '哆啦A梦：大雄的恐龙', '哑巴女人哑巴弹'],
       ['战斧骨(0)', '逃学威龙（1991-1993年周星驰主演系列电影）',
        '铃木商店的当家娘（根据玉冈薰的原作改编的剧情电影）'],
       ['死之花', '桑拿惊魂(2016)', '西部女郎'],
       ['布鲁克林规则', '全金属狂潮IntotheBlue(0)', '罗马风情 Gente di Roma(2003)'],
       ['Storyville(0)', '性的张力短片集(0)', '铜头铁罗汉'],
       ['飘香剑雨（1978年台湾电影）', '邻居（1981年郑洞天导演电影）', '流氓督察2'],
       ['小美人鱼2：重返大海', '砂之女', '在警局！(2018)'],
       ['灵猫 C.A.T(1987)', '飞越黄昏', '今夜你会不会来(0)'],
       ['康瑞德家的男孩', '旋转便利店', '沉底鳄'],
       ['快乐时光（电影）', '鲁邦三世：去死吧！诺斯特拉达穆斯(1995)', '魔发奇缘：幸福前奏(0)']],
      dtype='<U96')

#### Test Case 

In [49]:
import openai
from datetime import datetime
openai.api_key = 'YOUR_API_KEY'

result = openai.ChatCompletion.create(
          model="gpt-3.5-turbo-0301",
          messages=[
                {"role": "system", "content": "你帮助自动生成一个多轮次的电影推荐对话，每个发言以[用户]或[系统]开头，并利用提供的电影名作为输入。样例: 电影名 [十面埋伏], 生成对话: [用户]: 嗨，我正在寻找一部好电影看。你有什么推荐吗？[系统]: 当然！你有任何特定类型或题材的电影偏好吗？ [用户]: 我想看一部动作和历史背景的电影。有什么好的选择吗？[系统]: 嗯，如果你对中国历史感兴趣，我可以向你推荐一部叫做《十面埋伏》的电影。这是一部根据中国明朝时期的故事改编的电影，充满了动作和紧张的情节。[用户]: 听起来不错！请告诉我更多关于《十面埋伏》的信息。[系统]: 《十面埋伏》是由中国导演陈凯歌执导的一部史诗动作片。它以明朝末年的战争为背景，讲述了一个关于忍者的故事。影片中，主人公是一支特殊的刺客队伍，他们被派往对抗入侵的敌人。这部电影融合了动作、悬疑和历史元素，剧情扣人心弦，同时也展现了精彩的武术打斗场面。[用户]：太棒了！这个推荐都很适合我。谢谢你的帮助！[系统]：不客气！我很高兴能帮助你找到适合你口味的电影。希望你享受这些推荐，如果你还有其他问题，随时告诉我！"}, 
                {"role": "user", "content": "电影名：['铜头铁罗汉']"},
          ],
          stream=True
        )

In [50]:
completion_text = ''
collected_messages = []
for chunk in result:
    chunk_message = chunk['choices'][0]['delta']
    collected_messages.append(chunk_message)
full_reply_content = ''.join([m.get('content', '') for m in collected_messages])

In [51]:
full_reply_content

'[用户]: 你好，我正在找一部好看的电影，你有什么推荐吗？  \n[系统]: 当然，你有什么电影类型偏好吗？  \n[用户]: 我比较喜欢犯罪题材的电影。  \n[系统]: 那我可以向你推荐一部叫做《铜头铁罗汉》的电影。这是一部充满动作和犯罪元素的电影。  \n[用户]: 嗯，听起来不错，请你介绍一下这部电影。  \n[系统]: 《铜头铁罗汉》是由香港导演刘伟强执导的电影，讲述了一个黑帮团伙街头权利之争的故事。影片中，主人公们之间的斗争非常精彩，而主演铜头和铁罗汉两个角色的表现也非常出色。  \n[用户]: 这个推荐听起来非常不错！你能告诉我更多关于这部电影的信息吗？  \n[系统]: 当然可以。这部电影拥有一些很棒的演员阵容，包括谷峰、姜皓文和刘家辉等。尤其是主演铜头和铁罗汉两个角色，他们的表现非常精彩，同时影片的动作场景和镜头运用也非常出色，会让你感到非常刺激和热血。  \n[用户]: 好的，我会去看看这部电影的。谢谢你的推荐！  \n[系统]: 不用谢，希望你能够喜欢这部电影。如果你还有其他问题，可以随时问我。'

In [37]:
with open("log_5.json", "w") as f:
    json.dump(result, f, ensure_ascii=False)

### Dialog Generation

In [ ]:
import openai
from datetime import datetime
from tqdm import tqdm

openai.api_key = 'YOUR_API_KEY'

def process_str(movie_link):
    return ' '.join(movie_link[29:-1].split('_'))

results = []
unsuccessful_input = []
count = 0
success = 0
for movie in tqdm(movie_list):
    try:
        movie_names = ' '.join(movie)
        # movie = [process_str(link) for link in movie]
        # print("movie: [" + movie + "]")
        result = openai.ChatCompletion.create(
          model="gpt-3.5-turbo-0301",
          messages=[
                {"role": "system", "content": "你帮助自动生成一个多轮次的电影推荐对话，每个发言以[用户]或[系统]开头，并利用提供的电影名作为输入。样例: 电影名 [十面埋伏], 生成对话: [用户]: 嗨，我正在寻找一部好电影看。你有什么推荐吗？[系统]: 当然！你有任何特定类型或题材的电影偏好吗？ [用户]: 我想看一部动作和历史背景的电影。有什么好的选择吗？[系统]: 嗯，如果你对中国历史感兴趣，我可以向你推荐一部叫做《十面埋伏》的电影。这是一部根据中国明朝时期的故事改编的电影，充满了动作和紧张的情节。[用户]: 听起来不错！请告诉我更多关于《十面埋伏》的信息。[系统]: 《十面埋伏》是由中国导演陈凯歌执导的一部史诗动作片。它以明朝末年的战争为背景，讲述了一个关于忍者的故事。影片中，主人公是一支特殊的刺客队伍，他们被派往对抗入侵的敌人。这部电影融合了动作、悬疑和历史元素，剧情扣人心弦，同时也展现了精彩的武术打斗场面。[用户]：太棒了！这个推荐都很适合我。谢谢你的帮助！[系统]：不客气！我很高兴能帮助你找到适合你口味的电影。希望你享受这些推荐，如果你还有其他问题，随时告诉我！"}, 
                {"role": "user", "content": "电影名：[" + movie + "]"},
            ],
          stream=True
        )
        completion_text = ''
        collected_messages = []
        full_reply_content = ''.join([m['choices'][0]['delta'].get('content', '') for m in result])
        # for chunk in result:
        #     chunk_message = chunk['choices'][0]['delta']
        #     collected_messages.append(chunk_message)
        # full_reply_content = ''.join([m.get('content', '') for m in collected_messages])
        results.append({movie_names:full_reply_content}) 
        # print("Success ", count, " ", datetime.now())
        success += 1
        # print("Total Success: ", success)
    except Exception as e:
        # print("Error: ", e)
        # print("Problematic input: ", count, movie)
        unsuccessful_input.append(movie_names)
    if count % 1000 == 0 and count != 0:
        with open('output_dialog_tgredial_'+ str(count) +'.json', 'w') as outfile:
            json.dump(results, outfile, ensure_ascii=False)
        results = []
    count += 1

# Speed up the generation with multiprocess

In [23]:
from p_tqdm import p_map
import multiprocessing
import openai
from datetime import datetime
from tqdm import tqdm

openai.api_key = 'YOUR_API_KEY'

def dialogue_generation(movie):
    result = None
    while result is None:
        try:
            # print("processing: ", movie)
            result = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-0301",
              messages=[
                    {"role": "system", "content": "你帮助自动生成一个多轮次的电影推荐对话，每个发言以[用户]或[系统]开头，并利用提供的电影名作为输入。样例: 电影名 [十面埋伏], 生成对话: [用户]: 嗨，我正在寻找一部好电影看。你有什么推荐吗？[系统]: 当然！你有任何特定类型或题材的电影偏好吗？ [用户]: 我想看一部动作和历史背景的电影。有什么好的选择吗？[系统]: 嗯，如果你对中国历史感兴趣，我可以向你推荐一部叫做《十面埋伏》的电影。这是一部根据中国明朝时期的故事改编的电影，充满了动作和紧张的情节。[用户]: 听起来不错！请告诉我更多关于《十面埋伏》的信息。[系统]: 《十面埋伏》是由中国导演陈凯歌执导的一部史诗动作片。它以明朝末年的战争为背景，讲述了一个关于忍者的故事。影片中，主人公是一支特殊的刺客队伍，他们被派往对抗入侵的敌人。这部电影融合了动作、悬疑和历史元素，剧情扣人心弦，同时也展现了精彩的武术打斗场面。[用户]：太棒了！这个推荐都很适合我。谢谢你的帮助！[系统]：不客气！我很高兴能帮助你找到适合你口味的电影。希望你享受这些推荐，如果你还有其他问题，随时告诉我！"}, 
                    {"role": "user", "content": "电影名：[" + movie + "]"},
                ],
              stream=True
            )
            completion_text = ''
            collected_messages = []
            full_reply_content = ''.join([m['choices'][0]['delta'].get('content', '') for m in result])
            result = {movie:full_reply_content}
            return {movie:full_reply_content}
        except Exception as e:
            print("Error: ", e)
            print("Problematic input: ", movie)
            pass

if __name__ == '__main__':
    # pool = multiprocessing.Pool(processes=10)
    results = p_map(dialogue_generation, missing_generation, num_cpus=20)
    # pool.close()
    # pool.join()
    
    # with open('output_dialog_tgredial.json', 'w') as outfile:
    #     json.dump(results_2, outfile, ensure_ascii=False)
    
    # with open('results.txt', 'w') as file:
    #     for result in results:
    #         file.write(str(result) + '\n')

  0%|          | 0/24 [00:00<?, ?it/s]

Error:  That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 89d0b6f35a917f88eb4fee3dace85ec2 in your message.)
Problematic input:  穿越西伯利亚


## Address the missing generation

In [21]:
generated_results = json.load(open('output_dialog_tgredial_updated.json', 'r'))
missing_generation = []
for dialogue in generated_results:
    if list(dialogue.values())[0] == 'Failed':
        missing_generation.append(list(dialogue.keys())[0])

In [24]:
for dialogue in results:
    if dialogue == None:
        print(dialogue)
        continue
    for ori_dialogue in generated_results:
        if dialogue.keys() == ori_dialogue.keys():
            ori_dialogue[list(dialogue.keys())[0]] = list(dialogue.values())[0]

In [25]:
with open('output_dialog_tgredial_final.json', 'w') as outfile:
    json.dump(generated_results, outfile, ensure_ascii=False)

In [13]:
print(len(missing_generation))

1260


In [7]:
len(results)

31985

In [56]:
with open('output_dialog_tgredial_'+ str(count) +'.json', 'w') as outfile:
    json.dump(results, outfile, ensure_ascii=False)

In [ ]:
with open('output_dialog_tgredial_100_test.json', 'w') as outfile:
    json.dump(results, outfile, ensure_ascii=False)

### Identify Missing Examples

In [105]:
data = json.load(open('output_dialog_missing_1.json', 'r'))

broken_instance = []
for conv_idx, instance in enumerate(data):
    if conv_idx % 100 == 0:
        print("progress: ", conv_idx / len(data) * 100, "%", datetime.now())
    broken_dialog = False
    utterances = list(instance.values())[0]['choices'][0]['message']['content'].split('\n')
    utterances = list(filter(None, utterances))
    for idx, utterance in enumerate(utterances):
        utterance = utterance.strip()
        if utterance[:6].lower() == '[user]':
            continue
        elif utterance[:8].lower() == '[system]':
            continue
        else:
            broken_dialog = True
        if broken_dialog:
            print(utterances)
            break
    # if not broken_dialog:
    #     movies = list(instance.keys())[0]
    #     for movie in movies.split(' '):
    #         if movie not in movie_list:
    #             print(movie)
    #         else:
    #             movie_list.remove(movie)
    # else:
    #     continue
    if broken_dialog:
        broken_instance.append(list(instance.keys())[0])
        del data[conv_idx]

progress:  0.0 % 2023-04-30 13:13:03.989238
progress:  21.008403361344538 % 2023-04-30 13:13:03.990242
progress:  42.016806722689076 % 2023-04-30 13:13:03.991147
progress:  63.02521008403361 % 2023-04-30 13:13:03.992053
progress:  84.03361344537815 % 2023-04-30 13:13:03.992941


In [106]:
len(data)

476

In [103]:
broken_instance

[]

In [97]:
with open('output_dialog_saved_2.json', 'w') as outfile:
    json.dump(data, outfile)

In [86]:
broken_instance

'<http://dbpedia.org/resource/The_Frozen_Ground> <http://dbpedia.org/resource/Jeremiah_Johnson_(film)> <http://dbpedia.org/resource/Beyond_the_Law_(1992_film)>'

In [79]:
len(movie_list)

1429

### Generate Missing Instances

In [80]:
import random
random.shuffle(movie_list)
movies = np.asarray(movie_list[:1428]).reshape(476, 3)

In [ ]:
import openai
from datetime import datetime
openai.api_key = 'YOUR_API_KEY'

def process_str(movie_link):
    return ' '.join(movie_link[29:-1].split('_'))

results = []
unsuccessful_input = []
count = 0
success = 0
for movie in movies:
    try:
        movie_links = ' '.join(movie)
        movie = [process_str(link) for link in movie]
        print("movie list: [" + ', '.join(movie) + "]")
        result = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
                {"role": "system", "content": "You help in generating a mix-initiative synthetic multi-turn conversational recommendation dialog, each utterance starts with [user] or [system], by leveraging a list of mentioned movies as input.  Example: movie list: [Don't Look Up (2021), The Lost Daughter (2021)] dialog: [user]: Hi, can you suggest me some movies to watch? [system]: what kinds of movies do you like? [user]: Some recent popular movies could be considered. [system]: How about Don't Loop Up, which is quite popular and widely discussed. [user]: I have heard about it, but it is not what I would be interested. [system]: How about the lost daughter instead? [user]: What is it about? [system]: It is about a woman Leda Caruso who while on a summer holiday, finds herself becoming obsessed with another woman and her daughter on the beach. [user]: sounds interesting. Thanks for recommendation. [system]: You are welcome."}, 
                {"role": "user", "content": "movie list: [" + ', '.join(movie) + "]"},
            ]
        )
        results.append({movie_links:result}) 
        print("Success ", count, " ", datetime.now())
        success += 1
        print("Total Success: ", success)
    except Exception as e:
        print("Error: ", e)
        print("Problematic input: ", count, movie)
        unsuccessful_input.append(movie_links)
    count += 1

In [82]:
with open('output_dialog_missing_1.json', 'w') as outfile:
    json.dump(results, outfile)